# Problem Set 3
Designed by Sarah Adel Bargal, Ben Usman, and Kun He, with help from Kate Saenko and Brian Kulis.

This assignment will introduce you to:
- building, training, and testing of neural networks in TensorFlow
- using TensorFlow on the SCC cluster
- extracting and visualizing neural network features

Note: For programming, the use of any additional abstraction of TensorFlow (like `Keras` and `tf.contrib.learn`) is **not permitted**.

## Part 0: Tutorials
The contents of these tutorials are helpful for solving this assignment and upcoming assignments. Please be sure to go through them before proceeding.
1. [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/)
2. [Deep MNIST for Experts](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/)
3. [TensorFlow Mechanics 101](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/tf/)
4. [Sharing Variables](https://www.tensorflow.org/programmers_guide/variable_scope)

## Part 1: MNIST Softmax Classifier Demo in TensorFlow

TensorFlow is already installed on the SCC. Please review the instructions on [connecting to SCC](https://docs.google.com/document/d/1C4XrrTZIRfmJ6-LHmuJ4levTnunezujVrxCz8jJ4rec), and [running jobs on SCC](http://www.bu.edu/tech/support/research/system-usage/running-jobs/).

Make sure you are capable of running this demo (using `qsub`) on the SCC cluster: [`mnist_softmax_scope.py`](https://github.com/kunhe/cs591s2/blob/master/mnist_softmax_scope.py). There is no required deliverable, but this exercise is helpful for running jobs on the SCC in the future.

The demo is another implementation of `mnist_softmax.py` presented in the MNIST for ML Beginners tutorial, but using scopes. For the purposes of this assignment, the contents of the demo is also replicated below.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

def logistic_regression(x_):
    # create the actual model
    scope_args = {'initializer': tf.random_normal_initializer(stddev=1e-4)}
    with tf.variable_scope("weights", **scope_args):
        W = tf.get_variable('W', shape=[784, 10])
        b = tf.get_variable('b', shape=[10])
        y_logits = tf.matmul(x_, W) + b
    return y_logits

def test_classification(model_function, learning_rate=0.1):
    # import data
    mnist = read_data_sets('./datasets/mnist/', one_hot=True)

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/cpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, 784])
            y_ = tf.placeholder(tf.float32, [None, 10])
            y_logits = model_function(x_)
            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            losses = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_logits)
            cross_entropy_loss = tf.reduce_mean(losses)
            train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)
            
            y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            correct_prediction = tf.equal(y_pred, tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        sess.run(tf.initialize_all_variables())
        
        # train
        for iter_i in range(20001):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            sess.run(train_step, feed_dict={x_: batch_xs, y_: batch_ys})
            
            # test trained model
            if iter_i % 2000 == 0:
                tf_feed_dict = {x_: mnist.test.images, y_: mnist.test.labels}
                acc_value = sess.run(accuracy, feed_dict=tf_feed_dict)
                print('iteration %d\t accuracy: %.3f'%(iter_i, acc_value))
                
test_classification(logistic_regression, learning_rate=0.1)


## Part 2: Building Neural Networks in TensorFlow

(45 points)

### Q2.1 MLP in TensorFlow

Task: 

- Implement a multi-layer perceptron function **`mlp(x, hidden_sizes, activation_fn)`** in TensorFlow that has the following input and output: 

 **Inputs**
 - `x`, an input tensor of the images in the current batch `[batch_size, 28x28]`
 - `hidden_sizes`, a list of the number of hidden units per layer. For example: `[5,2]` means  5 hidden units in the first layer, and 2 hidden units in the second (output) layer.
 (Note: for MNIST, we need `hidden_sizes[-1]==10` since it has 10 classes.)
 - `activation_fn`, the activation function to be applied

 **Output**
 - a tensor of shape `[batch_size, hidden_sizes[-1]]`. 

**Note: ** 
- Make sure the activation function is not applied to the final (output) layer.
- It is recommended to use scopes and `tf.get_variable()` (as opposed to `tf.Variable()` which you may see elsewhere), as demonstrated in the sample code, and explained in the "Sharing Variables" tutorial in Part 0. Also see [here](http://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow).

In [ ]:
def mlp(x, hidden_sizes, activation_fn=tf.nn.relu):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
        
    ###################################
    ####     PUT YOUR CODE HERE    ####
    ###################################
    

The following code tests your  implementation of the **`mlp()`** function. It basically recreates the 2-layer MLP with ReLU activation that you implemented in Problem Set 2. It should give an accuracy of above 0.97.

In [ ]:
test_classification(lambda x: mlp(x, [64, 10], activation_fn=tf.nn.relu), learning_rate=0.1)

### Q2.2 Siamese Network
A siamese network is a network having two identical subnetworks that share parameters. A siamese network gives a *similarity metric* between pairs of inputs, which assigns high values to similar pairs and low values to dissimilar pairs. In MNIST, we can construct similar pairs by taking images from the same digit class, and dissimilar pairs from images from different classes. 


A sample Siamese network is presented below [Koch et al., Siamese Neural Networks for One-shot Image Recognition, ICML 2015]. Note how the two subnetworks have identical parameters:

<img src="https://lh4.googleusercontent.com/UoydUw_S34EuG6Fzv8Zxaqz-m8M-JsBr3tiADTricfpIFQSYz_jurZP-H1P-8ZKnHCO8xKeq-F_nErk=w1278-h954" style="width:480px;">


Given a Siamese network, we can determine the similarity between inputs $x_1$ and $x_2$, by taking the sign of the cosine similarity $\frac{\langle r_1,r_2\rangle}{\|r_1\|\|r_2\|}$, where $r_1$ and $r_2$ are the hidden representations produced by the network for $x_1$ and $x_2$, respectively. 

Tasks:

1. Build a Siamese Network in TensorFlow that receives as input two MNIST images, and decides whether they belong to the same digit category. Each subnetwork is an MLP with 2 hidden layers with 64 units, followed by a "distance layer" with 32 units. They will be created using the **`mlp()`** function you implemented earlier, using input argument `hidden_sizes=[64,64,32]`. The network computes the cosine similarity, as defined above, in the final output layer.

2. Train and test this model, and  report your test accuracy for this task.

You will need to implement the following two functions.

In [ ]:
#################################################################
# Inputs: arr1 and arr2 have shape [batch_size, hidden_sizes[-1]]
# Output: return tensor of shape [batch_size, ], the cosine 
#         similarity betwwen arr1 and arr2
# 
# Hint: use tf.l2_normalize, tf.mul, tf.reduce_sum
#################################################################
def cosine_similarity(arr1, arr2):
    ###################################
    ####     PUT YOUR CODE HERE    ####
    ###################################
    
    
#################################################################
# Inputs: mlp_args is a dictionary of arguments to the mlp() 
#         function. 
#         Example: mlp_args = {'hidden_sizes':[64, 64, 32]}
#################################################################
def build_model(mlp_args):
    with tf.Graph().as_default() as g:
        with tf.device("/cpu:0"):
            x1 = tf.placeholder(tf.float32, [None, 784])
            x2 = tf.placeholder(tf.float32, [None, 784])
            y = tf.placeholder(tf.float32, [None])

            with tf.variable_scope("siamese") as var_scope:
                x_repr1 = mlp(x1, **mlp_args)  # hidden representation of x1
                var_scope.reuse_variables()    # weight sharing! 
                x_repr2 = mlp(x2, **mlp_args)  # hidden representation of x2
                logits = cosine_similarity(x_repr1, x_repr2)  # similarity
                
            ###################################
            ####     PUT YOUR CODE HERE    ####
            ###################################
            
            # define scalar: loss 
            # define vector: y_prob as sigmoid(cosine_similarity)
            # define vector: y_pred as sign(cosine_similarity)
            # define scalar: accuracy as the fraction of correct predictions
            
    return {'graph': g, 'inputs': [x1, x2, y], 'pred': y_pred, 'logits': logits,
            'prob': y_prob, 'loss': loss, 'accuracy': accuracy}

The following code segment prepares minibatch training data.

In [ ]:
# data preparation
def mnist_siamese_dataset_iterator(batch_size, dataset_name):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1 # -1 for entire dataset
    mnist = input_data.read_data_sets('./datasets/mnist/', one_hot=True)
    dataset = getattr(mnist, dataset_name)
    
    while True:
        if batch_size > 0:
            X1, y1 = dataset.next_batch(batch_size)
            X2, y2 = dataset.next_batch(batch_size)
            y = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
            yield X1, X2, y
        else:
            X1 = dataset.images
            idx = np.arange(len(X1))
            np.random.shuffle(idx)
            X2 = X1[idx]
            y1 = dataset.labels
            y2 = y1[idx]
            y = np.argmax(y1, axis=1) == np.argmax(y2, axis=1)
            yield X1, X2, y

Now let's run the training.

Note: if you have a good GPU, you could change `tf.device("/cpu:0")` to `tf.device("/gpu:0")` in your **`build_model()`** function to speed up training.

In [ ]:
try:
    from itertools import izip as zip
except ImportError:
    print('This is Python 3')

def run_training(model_dict, train_data_iterator, test_full_iter, 
                 train_full_iter, n_iter=1000, print_every=100):
    with model_dict['graph'].as_default():
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
        train_op = optimizer.minimize(model_dict['loss'])
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for iter_i, data_batch in zip(range(n_iter), train_data_iterator):
                batch_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                sess.run(train_op, feed_dict=batch_feed_dict)
                if iter_i % print_every == 0:
                    print_zip_iter = zip([test_full_iter, train_full_iter], ['test', 'train'])
                    for data_iterator, data_name in print_zip_iter:
                        test_batch = next(data_iterator)
                        batch_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                        to_compute = [model_dict['accuracy'], model_dict['loss']]
                        acc_value, loss_val = sess.run(to_compute, batch_feed_dict)
                        fmt = (iter_i, acc_value, loss_val)
                        print(data_name, 'iteration %d\t accuracy: %.3f, loss: %.3f'%fmt)

train_data_iterator = mnist_siamese_dataset_iterator(100, 'train')
test_full_iter = mnist_siamese_dataset_iterator(-1, 'test')
train_full_iter = mnist_siamese_dataset_iterator(-1, 'train')

mlp_args = {'hidden_sizes':[64, 64, 32]}
model = build_model(mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter)

## Part 3: Model Variants

(30 points)

Tasks:
 
 1. Create an improved version of the model in part 2. You are welcome to use techniques covered in class, including: increasing network depth, varying activation functions, learning rates, optimizers, cost functions, regularization etc. 
 
 2. Report the test accuracy.

Note: you should **not** use convolutional layers for this assignment.

In [ ]:
def my_build_model(mlp_args):
    
    ###################################
    ####     PUT YOUR CODE HERE    ####
    ###################################
    
    return {'graph': g, 'inputs': [x1, x2, y], 'pred': y_pred, 'logits': logits,
            'prob': y_prob, 'loss': loss, 'accuracy': accuracy}

my_mlp_args = {}  # you can define your own arguments
model = my_build_model(my_mlp_args)
run_training(model, train_data_iterator, test_full_iter, train_full_iter)

## Part 4: Visualize learned features
(25 points)

Once a neural network is trained, we can think of the last layer as performing prediction, and the activations from layer before  become the input "features" to the predictor. If the neural network is performing good predictions, then this means that these activations encode useful features that are 1) representative of the input, and 2) discriminative for the prediction task. Activations at a selected layer could then be used as generic feature encodings of the input. 

We  expect a "good" feature encoding scheme to group similar inputs together in the feature encoding space. To check that, we can visualize the features in 2-dimensional space and check if similar examples (in this case, sharing the same labels) are close together. 

Tasks:

1. In TensorFlow, extract 32-dimensional features from the **distance layer** of your trained Siamese network, for the MNIST **test set**.
2. Reduce the dimensionality of your deep features to 2 using [t-SNE embedding](https://lvdmaaten.github.io/tsne/). You may use [this fast implementation](https://github.com/lvdmaaten/bhtsne/) with correct attribution. You could alternatively use [sklearn.manifold.TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), but please take note that it might be slow.
3. Visualize the 2-dimensional embeddings. If your extracted features are good, data points representing a specific digit should appear within a compact cluster. In the example below, each color corresponds to a digit class.

 <img src="https://lh3.googleusercontent.com/0mo16Fq5YrZhWWAscnBmxGbpui9RFrkMWhq3gqz0AkKW8c4omt94U_SSrK9siyiSF7dILzA9EO8WolQ=w1278-h954" style="width:480px;">
 
For this part, the starter code is in minimalist fashion. Good luck!

In [ ]:
def visualize(features, labels):
    ###################################
    ####     PUT YOUR CODE HERE    ####
    ###################################